In [1]:
import os
import numpy as np
import tensorflow as tf
from scipy.misc.pilutil import imread, imresize
import matplotlib.pyplot as plt
%matplotlib inline

/home/choiseokokyu/miniconda3/envs/deep/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# def img_to_data(path, img_size):

#    cwd = os.getcwd()
#    path = path
#    image_list = os.listdir(cwd+path)
#    image_array = []
#    label_array = []
#    for i in image_list:
#        img = imread(cwd+path+i)
#        resize_img = imresize(img, img_size)
#        image_array.append(resize_img)
#        if "cat" in i :
#            label_array.append("cat")
#        else:
#            label_array.append('dog')
#    print("check :", np.shape(image_array))
#    print("label :", np.shape(label_array))

#    split_value = int(0.8*(len(image_array)))
#    train_img = image_array[:split_value]
#    test_img = image_array[split_value:]
#    train_label = label_array[:split_value]
#    test_label = label_array[split_value:]

#    savepath = cwd+"/dataset.npz"
#    np.savez(savepath,train_img=train_img,train_label=train_label,
#            test_img=test_img,test_label=test_label)
#    print("data saved to %s"%(savepath))

# img_to_data('/train/', [256,256,3])

In [24]:
data = np.load("dataset.npz")
train_img = data["train_img"]
train_label = data["train_label"]
test_img = data["test_img"]
test_label = data["test_label"]

In [25]:
train_label = [[1,0] if i == "cat" else [0,1] for i in train_label]
test_label = [[1,0] if i =="cat" else [0,1] for i in test_label]
np.array(test_label).shape

(5000, 2)

In [5]:
training_epochs = 20
batch_size = 100
learning_rate = 0.01

In [6]:
X = tf.placeholder(tf.float32, shape=[None, 256, 256,3])
Y = tf.placeholder(tf.float32, shape=[None, 2])

keep_prob = tf.placeholder(tf.float32)

#conv layer 1
W1 = tf.get_variable("w1", shape = [11,11,3,64], dtype = tf.float32, initializer=tf.contrib.layers.xavier_initializer())
conv1 = tf.nn.conv2d(X, W1, strides=[1,4,4,1], padding="SAME")
b1 = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32), trainable = True, name='b1')
conv1 = tf.nn.relu(tf.nn.bias_add(conv1, b1))
lrn1 = tf.nn.local_response_normalization(conv1, alpha = 1e-4, beta = 0.75, depth_radius=2, bias=2.0)
max_pool1 = tf.nn.max_pool(lrn1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID')


#conv layer 2
W2 = tf.get_variable("w2", shape = [5, 5, 64, 192], dtype = tf.float32, initializer=tf.contrib.layers.xavier_initializer())
conv2 = tf.nn.conv2d(max_pool1, W2, strides=[1,1,1,1], padding='SAME')
b2 = tf.Variable(tf.constant(0.0, shape=[192], dtype = tf.float32), trainable=True, name='b2')
conv2 = tf.nn.relu(tf.nn.bias_add(conv2, b2))
lrn2 = tf.nn.local_response_normalization(conv2, alpha = 1e-4, beta=0.75, depth_radius =2, bias=2.0)
max_pool2 = tf.nn.max_pool(lrn2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID')


#conv layer 3
W3 = tf.get_variable("w3", shape = [3, 3 , 192, 384], dtype = tf.float32, initializer=tf.contrib.layers.xavier_initializer())
conv3 = tf.nn.conv2d(max_pool2, W3, strides=[1,1,1,1], padding='SAME')
b3 = tf.Variable(tf.constant(0.0, shape=[384], dtype = tf.float32), trainable=True, name='b3')
conv3 = tf.nn.relu(tf.nn.bias_add(conv3, b3))

#conv  layer 4
W4 = tf.get_variable("w4", shape = [3, 3 , 384, 256], dtype = tf.float32, initializer=tf.contrib.layers.xavier_initializer())
conv4 = tf.nn.conv2d(conv3, W4, strides=[1,1,1,1], padding='SAME')
b4 = tf.Variable(tf.constant(0.0, shape=[256], dtype = tf.float32), trainable=True, name='b4')
conv4 = tf.nn.relu(tf.nn.bias_add(conv4, b4))

#conv layer 5
W5 = tf.get_variable("w5", shape = [3, 3 , 256, 256], dtype = tf.float32, initializer=tf.contrib.layers.xavier_initializer())
conv5 = tf.nn.conv2d(conv4, W5, strides=[1,1,1,1], padding='SAME')
b5 = tf.Variable(tf.constant(0.0, shape=[256], dtype = tf.float32), trainable=True, name='b5')
conv5 = tf.nn.relu(tf.nn.bias_add(conv5, b5))
max_pool5 = tf.nn.max_pool(conv5, ksize=[1,3,3,1], strides=[1,2,2,1], padding='VALID')

#fully_connected1
flat = tf.reshape(max_pool5, [-1, 7*7*256])
W6 = tf.get_variable("w6", shape = [7*7*256, 4096], dtype = tf.float32, initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.get_variable("b6", shape = [4096], dtype = tf.float32, initializer=tf.contrib.layers.xavier_initializer())
fc6 = tf.nn.relu(tf.nn.bias_add(tf.matmul(flat,W6), b6))
fc6 = tf.nn.dropout(fc6, keep_prob=keep_prob)

#fully_connected2
W7 = tf.get_variable("w7", shape = [4096,4096], dtype = tf.float32, initializer=tf.contrib.layers.xavier_initializer())
b7 = tf.get_variable("b7", shape = [4096], dtype = tf.float32, initializer=tf.contrib.layers.xavier_initializer())
fc7 = tf.nn.relu(tf.nn.bias_add(tf.matmul(fc6,W7), b7))
fc7 = tf.nn.dropout(fc7, keep_prob=keep_prob)

#fully_connected3
W8 = tf.get_variable("w8", shape = [4096,2], dtype = tf.float32, initializer=tf.contrib.layers.xavier_initializer())
b8 = tf.get_variable("b8", shape = [2], dtype = tf.float32, initializer=tf.contrib.layers.xavier_initializer())
logits = tf.matmul(fc7, W8) + b8

cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Learning Started")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(train_img)/batch_size)
    for i in range(total_batch):print(max_pool5)
        batch_xs = train_img[i*batch_size:(i+1)*batch_size]
        batch_ys = train_label[i*batch_size:(i+1)*batch_size]
        c, _ = sess.run([cost, optimizer], feed_dict = {X : batch_xs, Y: batch_ys, keep_prob:0.7})
        avg_cost += c/total_batch
    
    print("Epoch :", epoch+1, "cost :", avg_cost)
    
print("Learning Finished")

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy :", sess.run(accuracy, feed_dict={X : test_img, Y: test_label, keep_prob:1}))

Learning Started
Epoch : 1 cost : 403676.90941391577
Epoch : 2 cost : 0.7102384680509564
Epoch : 3 cost : 0.7012353065609933
Epoch : 4 cost : 0.703259896636009
Epoch : 5 cost : 0.6931400135159492
Epoch : 6 cost : 0.6931713408231738
Epoch : 7 cost : 0.6936751604080204
Epoch : 8 cost : 0.693328785300255
Epoch : 9 cost : 0.6934324809908866
Epoch : 10 cost : 0.6933325231075288
Epoch : 11 cost : 0.6931585282087328
Epoch : 12 cost : 0.693118034899235
Epoch : 13 cost : 0.6931599214673044
Epoch : 14 cost : 0.6931277531385425
Epoch : 15 cost : 0.693127153217792
Epoch : 16 cost : 0.6931413248181345
Epoch : 17 cost : 0.6931285789608959
Epoch : 18 cost : 0.6931276512145995
Epoch : 19 cost : 0.6930949181318287
Epoch : 20 cost : 0.6931133279204367
Learning Finished


InvalidArgumentError: Incompatible shapes: [5000] vs. [20000]
	 [[Node: Equal = Equal[T=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ArgMax, ArgMax_1)]]

Caused by op 'Equal', defined at:
  File "/home/choiseokokyu/miniconda3/envs/deep/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/choiseokokyu/miniconda3/envs/deep/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/choiseokokyu/miniconda3/envs/deep/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/choiseokokyu/miniconda3/envs/deep/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/choiseokokyu/miniconda3/envs/deep/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/choiseokokyu/miniconda3/envs/deep/lib/python3.6/site-packages/zmq/event

In [26]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
ac = sess.run(accuracy, feed_dict={X : test_img, Y: test_label, keep_prob:1})
print("Accuracy :", ac)

Accuracy : 0.502


In [22]:
train
0~255/ 0~1 ... 1


(5000, 2)
